In [ ]:
!pip install pytesseract
!sudo apt install tesseract-ocr -y
!sudo apt install tesseract-ocr-ara tesseract-ocr-hin tesseract-ocr-nep tesseract-ocr-fas -y


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tesseract-ocr-ara tesseract-ocr-fas tesseract-ocr-hin tesseract-ocr-nep
0 upgraded, 4 newly installed, 0 to remove and 38 not upgraded.
Need to get 2,336 kB of archives.
After this operation, 4,049 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-ara all 1:4.00~git30-7274cfa-1.1 [645 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-fas all 1:4.00~git30-7274cfa-1.1 [301 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-hin all 1:4.00~git30-7274cfa-1.1 [913 kB]
Get:4 http://archive.ubuntu.com/ubuntu jamm

In [ ]:
import os
import json
import torch
import cv2
import numpy as np
from scipy.ndimage import rotate
import pytesseract
from PIL import Image, ImageDraw, ImageFont
from transformers import DetrImageProcessor, DetrForObjectDetection

# ---------------- Settings ---------------- #
model_path = "/content/drive/MyDrive/layout_detection_finetuned"
dataset_dir = "/content/drive/MyDrive/Data for PS-05 Intelligent Multi Lingual Document/Mock Data"
output_dir = "/content/drive/MyDrive/mock_data_output1"

# Subdirectories for outputs
output_img_dir = os.path.join(output_dir, "output_images")
output_json_dir = os.path.join(output_dir, "output_json")
os.makedirs(output_img_dir, exist_ok=True)
os.makedirs(output_json_dir, exist_ok=True)

# Map class ids (0-based from model) to dataset ids + names
id2label = {
    0: "text",
    1: "title",
    2: "list",
    3: "table",
    4: "figure"
}

# ------------------ Tesseract Setup ------------------ #
pytesseract.pytesseract.tesseract_cmd = r"/usr/bin/tesseract"
LANGS = "eng+ara+hin+nep+fas"
OCR_CONFIG = "--psm 6"

# ------------------ Deskew Logic ------------------ #
def correct_skew(image, delta=1, limit=5):
    def determine_score(arr, angle):
        rotated = rotate(arr, angle, reshape=False, order=0)
        hist = np.sum(rotated, axis=1, dtype=float)
        return np.sum((hist[1:] - hist[:-1]) ** 2, dtype=float)

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

    scores = []
    angles = np.arange(-limit, limit + delta, delta)
    for a in angles:
        scores.append(determine_score(thresh, a))

    best_angle = angles[scores.index(max(scores))]
    if abs(best_angle) < 0.5:
        print(f"✅ Image is already upright (angle={best_angle:.2f}°).")
        return best_angle, image  # no change

    print(f"🔧 Detected skew angle: {best_angle:.2f}° → correcting...")
    (h, w) = image.shape[:2]
    M = cv2.getRotationMatrix2D((w // 2, h // 2), best_angle, 1.0)
    corrected = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return best_angle, corrected

# ------------------ Orientation Correction Logic ------------------ #
def is_valid_multilingual_char(c: str) -> bool:
    code = ord(c)
    if (0x0041 <= code <= 0x007A) or (0x0030 <= code <= 0x0039): return True
    if 0x0900 <= code <= 0x097F: return True     # Devanagari
    if 0x0600 <= code <= 0x06FF or 0x0750 <= code <= 0x077F or 0xFB50 <= code <= 0xFEFF: return True
    if 0x0660 <= code <= 0x0669: return True
    if c in " .,:;!?।،؟-–—()[]{}%/\\\"'": return True
    return False

def compute_char_score(text: str) -> float:
    return sum(is_valid_multilingual_char(c) for c in text) / (len(text) + 1e-5)

def preprocess_for_ocr(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.fastNlMeansDenoising(gray, None, 10, 7, 21)
    return gray

def evaluate_image_variant(img):
    gray = preprocess_for_ocr(img)
    try:
        text = pytesseract.image_to_string(gray, lang=LANGS, config=OCR_CONFIG)
    except Exception:
        text = ""
    score = compute_char_score(text)
    return score, text

def rotate_img(image, angle: int):
    if angle == 0: return image
    if angle == 90: return cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
    if angle == 180: return cv2.rotate(image, cv2.ROTATE_180)
    if angle == 270: return cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)
    return image

def detect_best_orientation(image):
    rotations = [0, 90, 180, 270]
    flips = [None, 'H', 'V']
    best = {"score": -1, "angle": 0, "flip": None, "img": image, "text": ""}

    for flip in flips:
        for angle in rotations:
            rotated = rotate_img(image, angle)
            if flip == 'H': rotated = cv2.flip(rotated, 1)
            elif flip == 'V': rotated = cv2.flip(rotated, 0)
            score, text = evaluate_image_variant(rotated)
            label = f"{angle}°{' + Flip'+flip if flip else ''}"
            print(f"  {label:<10} → Score: {score:.3f}, len={len(text)}")
            if score > best["score"]:
                best.update({"score": score, "angle": angle, "flip": flip, "img": rotated, "text": text})

    flip_label = f" + Flip{best['flip']}" if best['flip'] else ""
    print(f"\n✅ Best orientation: {best['angle']}°{flip_label} (score={best['score']:.3f})")
    return best["img"]

def preprocess_image(image_path):
    print(f"\n📄 Preprocessing: {os.path.basename(image_path)}")
    img = cv2.imread(image_path)
    if img is None:
        print("❌ Could not read image.")
        return None
    angle, deskewed = correct_skew(img)
    final = deskewed if abs(angle) >= 0.5 else detect_best_orientation(deskewed)
    return final

# ---------------- Load model ---------------- #
device = "cuda" if torch.cuda.is_available() else "cpu"
processor = DetrImageProcessor.from_pretrained(model_path)
model = DetrForObjectDetection.from_pretrained(model_path).to(device)
model.eval()

# ---------------- Inference ---------------- #
def run_inference(image_path, corrected_img, detection_threshold=0.5):
    image = Image.fromarray(cv2.cvtColor(corrected_img, cv2.COLOR_BGR2RGB))
    encoding = processor(images=image, return_tensors="pt").to(device)
    outputs = model(**encoding)

    # Post-process predictions
    target_size = torch.tensor([image.size[::-1]])  # (height, width)
    results = processor.post_process_object_detection(
        outputs, target_sizes=target_size, threshold=detection_threshold
    )[0]

    draw = ImageDraw.Draw(image)
    font = ImageFont.load_default()

    annotations = []
    for bbox, score, label in zip(results["boxes"], results["scores"], results["labels"]):
        x_min, y_min, x_max, y_max = bbox.tolist()
        cls_id = int(label.item()) + 1  # make category_id 1-based
        cls_name = id2label.get(int(label.item()), "unknown")

        # Draw visualization
        draw.rectangle([x_min, y_min, x_max, y_max], outline="red", width=2)
        draw.text((x_min, y_min - 10), f"{cls_name} ({score:.2f})", fill="red", font=font)

        xyhw = [round(x_min, 2), round(y_min, 2), round(y_max - y_min, 2), round(x_max - x_min, 2)]

        annotations.append({
            "bbox": xyhw,
            "category_id": cls_id,
            "category_name": cls_name
        })

    output_json = {
        "file_name": os.path.basename(image_path),
        "annotations": annotations,
        "corruption": {"type": "none", "severity": 0}
    }

    # Save outputs
    base_name = os.path.basename(image_path).rsplit(".", 1)[0]
    json_path = os.path.join(output_json_dir, f"{base_name}.json")
    with open(json_path, "w") as f:
        json.dump(output_json, f, indent=4)

    vis_path = os.path.join(output_img_dir, f"{base_name}_vis.png")
    image.save(vis_path)

    print(f"✅ Processed {os.path.basename(image_path)} -> JSON + visualization saved.")


# ---------------- Batch Run ---------------- #
if __name__ == "__main__":
    image_files = [f for f in os.listdir(dataset_dir) if f.lower().endswith((".png", ".jpg", ".jpeg"))]

    print(f"Found {len(image_files)} images in {dataset_dir}")
    for img_file in image_files:
        img_path = os.path.join(dataset_dir, img_file)
        corrected = preprocess_image(img_path)
        if corrected is not None:
            run_inference(img_path, corrected, detection_threshold=0.5)

    print(f"\n🎯 All results saved to: {output_dir}")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/module.py:2441: UserWarning: for conv1.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/nn/modules/module.py:2441: UserWarning: for bn1.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/nn/modules/module.py:2441: UserWarning: for bn1.bias: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pas

Found 1000 images in /content/drive/MyDrive/Data for PS-05 Intelligent Multi Lingual Document/Mock Data

📄 Preprocessing: doc_00006 (1).png
✅ Image is already upright (angle=0.00°).
  0°         → Score: 0.966, len=3105
  90°        → Score: 0.839, len=834
  180°       → Score: 0.927, len=2324
  270°       → Score: 0.893, len=1097
  0° + FlipH → Score: 0.936, len=2124
  90° + FlipH → Score: 0.877, len=708
  180° + FlipH → Score: 0.921, len=1956
  270° + FlipH → Score: 0.873, len=652
  0° + FlipV → Score: 0.921, len=1956
  90° + FlipV → Score: 0.873, len=652
  180° + FlipV → Score: 0.936, len=2124
  270° + FlipV → Score: 0.877, len=708

✅ Best orientation: 0° (score=0.966)
✅ Processed doc_00006 (1).png -> JSON + visualization saved.

📄 Preprocessing: doc_00012.png
✅ Image is already upright (angle=0.00°).
  0°         → Score: 0.930, len=2501
  90°        → Score: 0.883, len=666


KeyboardInterrupt: 

In [2]:
import os

# Path to your folder (change this to your actual path)
folder_path = "/content/drive/MyDrive/PS05_SHORTLIST_DATA/images"

# Count only image files
image_extensions = (".jpg", ".jpeg", ".png", ".bmp", ".gif", ".tiff", ".webp")
num_images = sum(1 for file in os.listdir(folder_path) if file.lower().endswith(image_extensions))

print(f"📸 Number of images in '{folder_path}': {num_images}")


📸 Number of images in '/content/drive/MyDrive/PS05_SHORTLIST_DATA/images': 5285
